In [1]:
import feets.preprocess
from scipy import stats
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
import feets
from ipykernel import kernelapp as app
import os



script_dir = os.path.dirname('Results/')
results_dir = os.path.join(script_dir, 'Images/')

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

In [ ]:
url = "https://raw.githubusercontent.com/MachineLearningUniandes/MANTRA/master/data/lightcurves/transient_lightcurves.csv"
url1 = 'https://raw.githubusercontent.com/MachineLearningUniandes/MANTRA/master/data/lightcurves/transient_labels.csv'
url2 = 'https://raw.githubusercontent.com/MachineLearningUniandes/MANTRA/master/data/lightcurves/transient_info.txt'
transient_lc = pd.read_csv(url)
transient_labels = pd.read_csv(url1)
transient_info = pd.read_table(url2,skiprows=1,names=['CRTS_ID' ,'RA' ,'Dec','UT_Date','Mag','CSS_images' ,'SDSS',
                                                      'Others' ,'Followed' ,'Last','LC','FC','Classification'])

transient_info.head()


In [ ]:
import seaborn as sns
from sklearn.ensemble import IsolationForest

In [ ]:
# Reading in the feature.csv data
ft_data = pd.read_csv('features.csv',skiprows=1,names= ['Amplitude', 'AndersonDarling', 'Autocor_length', 'Beyond1Std',
                                                  'CAR_sigma', 'Con', 'Eta_e', 'Freq1_harmonics_rel_phase_1',
                                                  'LinearTrend', 'MaxSlope', 'Mean', 'Period_fit', 
                                                  'SlottedA_length','SmallKurtosis', 'StructureFunction_index_21'
                                                  ,'ID','Class'])


ft_data.head()

In [ ]:
params = ['Amplitude', 'AndersonDarling', 'Autocor_length', 'Beyond1Std',
                                                  'CAR_sigma', 'Con', 'Eta_e', 'Freq1_harmonics_rel_phase_1',
                                                  'LinearTrend', 'MaxSlope', 'Mean', 
                                                  'SlottedA_length','SmallKurtosis', 'StructureFunction_index_21'
                                                  ]

X = ft_data[params]

In [ ]:
model=IsolationForest(n_estimators=400, max_samples='auto',max_features=1.0)
x=model.fit(X)



pred_cls = []
# Predicting the anomalies
ft_data['iforest'] = pd.Series(model.predict(X))
ft_data['iforest_Counts'] = ft_data['iforest'].map({1:0,-1:1})
# Counting the detected anomalies
print(ft_data['iforest'].value_counts())

# Analysing the predicted anomalies
pred=ft_data.loc[ft_data['iforest'] == -1]
for i in range(len(pred.ID)):
    
    
#     plt.figure(figsize=(16,4))
#     plt.xlabel('MJD')
#     plt.ylabel('Mag')
    lc = transient_lc[transient_lc['ID']==pred.ID.iloc[i]]
    
#     print(len(lc),pred.Class.iloc[i],pred.ID.iloc[i])
    pred_cls.append(pred.Class.iloc[i])
#     plt.errorbar(lc.MJD,lc.Mag,yerr=lc.Magerr,fmt='.r',label=pred.Class.iloc[i])
#     plt.legend()
#     plt.show()

In [ ]:
sklearn_score_anomalies = model.decision_function(X)

# print(sklearn_score_anomalies)

x = [i  for i in sklearn_score_anomalies]

# len(ft_data[x])

anomalies_scr=sklearn_score_anomalies
anomalies_cls = ft_data.Class


# Getting the unique classes detected as anomalous
ML_unq_cls=np.unique(pred_cls)
# x = [i < 0 for i in sklearn_score_anomalies]
# anomalies_scr=sklearn_score_anomalies[x]
# anomalies_cls = ft_data.Class[x]
# anomalies_ID = ft_data.ID[x]

# Getting the number of objects in each anomalous class 
# based on the transient label data
len_anm = []
for i in ML_unq_cls:
    
    lb = transient_labels[transient_labels['Classification']==i]   
    len_anm.append(len(lb))


# Sorting the classes in ascending based on the number of
# objects in each class (i.e most anomalous to least anomalous)
cls = []  
for i in range(0,max(len_anm)+1): 
    
    for j in range(len(len_anm)): 
        
        if len_anm[j] == i: 
            cls.append(ML_unq_cls[j])
            
                        
# Calculating the average and std of the anom
# scores

sstd,aave = [],[] ; catgories = []; scr1=[]
for j in range(len(cls)):
    scr = []
    for i in range(len(anomalies_cls)):
    
        if anomalies_cls.iloc[i] == cls[j]:
            
            scr.append(anomalies_scr[i])
            scr1.append(anomalies_scr[i])
            
    catgories.append(scr)
            
#             print(anomalies_cls.iloc[i],anomalies_scr[i])
#     print(scr,np.std(scr))       
    std=np.std(scr)
    ave=np.average(scr)   
    sstd.append(std);aave.append(ave)
    
#     print(ave,std)
    
    

    
# Plotting the anomalous objects based on their score 
plt.rcParams.update({'font.size': 24})
plt.figure(figsize=(15,8))
df = pd.DataFrame({"id":cls, 
                   "score":aave,
                   'error':sstd
                   })
plt.errorbar(np.arange(len(df['id'])), df['score'],yerr=df['error'],fmt='-ro',ecolor='b',label='ave score',linewidth=2.2)
ax = plt.gca()
ax.xaxis.set_ticks(np.arange(len(df['id'])))
ax.xaxis.set_ticklabels(df['id'], rotation=90)
plt.xlabel("Class")
plt.ylabel("Anomaly_score")
plt.gca().invert_yaxis()
plt.legend()
plt.savefig(results_dir + 'ave_score_all.png',bbox_inches="tight")
plt.show()

In [ ]:
# import pandas as pd 
import pandas as pd 
  
# both lists, with columns specified 
df = pd.DataFrame(list(zip(anomalies_scr, anomalies_cls)), 
               columns =['anomaly_score', 'class']) 



plt.figure(figsize=(16,6))
x = sns.violinplot(x="class", y="anomaly_score", data=df,gridsize=200,order=cls,width=0.8)
plt.savefig('violin_plot.png')
# plt.xlabel('m')
plt.axhline(0,color='black',ls=':')
plt.xticks(rotation=45)
plt.show()